# comet
> Library to score machine translations with the COMET metric
>
> This library needs a CUDA-compatible GPU to execute (no check yet)

In [ ]:
#| default_exp comet

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import comet
from comet import download_model, load_from_checkpoint
from pathlib import Path
import csv
import os

class cometscoring:
    """Class to calculate COMET score (with references)"""
    def __init__(self,model_name="wmt20-comet-da",gpus=1):
        """Constructor that downloads and loads the COMET model"""
        model_path = download_model(model_name)
        self.model = load_from_checkpoint(model_path)
        self.gpus = gpus

    def measure_comet(self,source_lines,hypothesis_lines,reference_lines):
        """Function to calculate the comet score"""
        # Construct array necessary to measure COMET
        comet_data = []
        for (src,mt,ref) in zip(source_lines,hypothesis_lines,reference_lines):
            comet_dict = {"src":src,"mt":mt,"ref":ref}
            comet_data.append(comet_dict)
        # Run COMET
        seg_scores, sys_score = self.model.predict(comet_data, batch_size=8, gpus=self.gpus)

        return seg_scores, sys_score

    def measure_record_comet(self,source_lines,hypothesis_lines,reference_lines,sourcelang,targetlang,test_set_name,test_date,mtengine,score_pathname,score_fname,domain=''):
        """Function to score hypothesis with COMET score and record score to a specified metrics file"""           
        seg_comet_scores, sys_score = self.measure_comet(source_lines,hypothesis_lines,reference_lines)

        # Write the corpus COMET score with some meta data to CSV file
        metric_record = {}
        metric_record["name"] = "COMET"
        metric_record["score"] = sys_score
        metric_record["version"] = comet.__version__
        metric_record["date"] = test_date
        metric_record["source_langid"] = sourcelang
        metric_record["target_langid"] = targetlang            
        metric_record["test_set"] = test_set_name
        metric_record["engine"] = mtengine
        metric_record["domain"] = domain
        score_path = Path(score_pathname,score_fname)
        with open(score_path,"a",newline="") as score_file:
            field_names = metric_record.keys()
            writer = csv.DictWriter(score_file, fieldnames=field_names)
            if score_file.tell() == 0:
                writer.writeheader()
            writer.writerow(metric_record)   

        # Write segment level COMET scores to plain text file
        output_path = score_pathname+"/"+sourcelang+"_"+targetlang+"/"+test_date+"/"+test_set_name
        os.makedirs(output_path,exist_ok=True)
        seg_score_fname = ""
        if domain:
            seg_score_fname = "comet_"+mtengine+"."+domain+"."+sourcelang+"-"+targetlang
        else:
            seg_score_fname = "comet_"+mtengine+"."+sourcelang+"-"+targetlang
        seg_score_path = Path(output_path,seg_score_fname)
        with open(seg_score_path,"w") as seg_score_fh:
            for seg_comet_score in seg_comet_scores:
                print(seg_comet_score,file=seg_score_fh)


In [ ]:
#| hide
import os
import torch
from mteval.dataset import *
import json
from datetime import date

In [ ]:
#| hide
# Testing with GPU on Github Actions is tricky and therefore not implemented yet
# Unit test instead on Google Colab

if not os.getenv('GITHUB_ACTIONS') == "true" and torch.cuda.is_available():
    mteval_test_path = homedir+"/mteval_test/"
    source_seg, ref_seg = download_read_set(mteval_test_path,"en","it","wmt09")
    hyp_seg = ref_seg
    comet_scorer = cometscoring()
    seg_comet_scores, sys_comet = comet_scorer.measure_comet(source_seg,hyp_seg,ref_seg)
    # Only check a number of score results to verify a validly formatted result is returned
    assert sys_comet == 1.183892640986378
    assert seg_comet_scores[0] == 1.2579313516616821
    assert seg_comet_scores[10] == 1.142746090888977
    assert seg_comet_scores[100] == 1.3046232461929321
    assert seg_comet_scores[1000] == 1.2136374711990356
    assert len(seg_comet_scores) == 3027

    isodate = date.today().isoformat()
    score_filename = "comet_test.csv"
    comet_scorer.measure_record_comet(source_seg,hyp_seg,ref_seg,"en","it","wmt09",isodate,"dummy",mteval_test_path,score_filename)
    assert os.path.exists(mteval_test_path+score_filename) == 1
    assert os.path.exists(mteval_test_path+"en_it/"+isodate+"/wmt09/comet_dummy.en-it") == 1

In [ ]:
show_doc(cometscoring.measure_comet)

In [ ]:
show_doc(cometscoring.measure_record_comet)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()